In [1]:
# Imports
import requests

In [5]:
# Billing Status
from Helper import StakingRewardsAPIClient

client = StakingRewardsAPIClient()
status = client.get_billing_status()
print(status)

{'plan_name': 'startup yearly v0', 'available_credits': 1499934, 'monthly_quota': 1500000, 'subscribed_at': '2025-11-24T07:32:41Z', 'requests_allowed': True, 'billing_date': '2025-12-24T07:32:41Z'}


In [5]:
# Asset Queries
eth = client.get_assets(symbols=["ETH"], limit=1)
print(eth)

DEBUG - Query: 
        {
          assets(where: {symbols: ["ETH"]}, limit: 3) {
            id
            name
            slug
            description
            symbol
          }
        }
        
{'data': {'assets': [{'id': '5d27aa0de03dd80007d1eb49', 'name': 'Ethereum', 'slug': 'ethereum-2-0', 'description': "the world's largest and most decentralized Layer1 blockchain. The network is used for building dApps, holding assets, transacting and communicating without being controlled by a central authority. The Ethereum vision is to build a digital future on a global scale, that is powerful enough to help all of humanity", 'symbol': 'ETH'}]}}


In [4]:
result = client.get_asset_metrics(
      slug="polkadot",
      metric_keys=["reward_rate", "provider"],
      created_before="2023-06-28",
      metrics_limit=10
  )
print(result)

{'data': {'assets': [{'id': '5d27aa0ee03dd80007d1eb6d', 'slug': 'polkadot', 'logoUrl': 'https://coin-images.coingecko.com/coins/images/12171/large/polkadot.png?1696512008', 'metrics': [{'defaultValue': 14.835948403683611, 'createdAt': '2023-06-27T22:01:16.026146Z'}, {'defaultValue': 14.835916173737743, 'createdAt': '2023-06-27T20:01:07.394307Z'}, {'defaultValue': 14.835696505476331, 'createdAt': '2023-06-27T18:11:07.667837Z'}, {'defaultValue': 14.835516951370295, 'createdAt': '2023-06-27T16:01:13.578876Z'}, {'defaultValue': 14.852641094515928, 'createdAt': '2023-06-27T14:01:58.737205Z'}, {'defaultValue': 14.852593442400858, 'createdAt': '2023-06-27T12:10:17.547877Z'}, {'defaultValue': 14.852589686723764, 'createdAt': '2023-06-27T10:01:22.273959Z'}, {'defaultValue': 14.85250720223731, 'createdAt': '2023-06-27T08:01:23.624704Z'}, {'defaultValue': 14.8524890814507, 'createdAt': '2023-06-27T06:01:55.861665Z'}, {'defaultValue': 14.852486411180484, 'createdAt': '2023-06-27T04:01:27.988426Z'}